<a href="https://colab.research.google.com/github/shantam-8/ML-Materials-Individual-Project/blob/main/Individual_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Individual Project

In [ ]:
import numpy as np

# Insert your values
Name = "Shantam Gilra" # Replace with your name
CID = 1878023 # Replace with your College ID (as a numeric value with no leading 0s)

# Set a random seed using the CID value
CID = int(CID)
np.random.seed(CID)

# Print the message
print("This is the work of " + Name + " [CID: " + str(CID) + "]")

## Problem statement

You have been assigned one dataset from the [list](https://matbench.materialsproject.org/Benchmark%20Info/matbench_v0.1/) on [MatBench](https://matbench.materialsproject.org). You should state what problem you are trying to solve and comment on the best-performing model in the benchmark.

In [ ]:
# Spare cell




## Data preparation

Check the data distribution and apply appropriate pre-processing steps as required.

In [1]:
# Installation of libraries
!pip install matminer=="0.9.0" --quiet
!pip install pymatgen=="2023.09.25" --quiet
!pip install xgboost --quiet
!pip install elementembeddings --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.7/807.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# !pip install CBFV --user

# !pip install -U xenonpy

# !pip install parallel-pandas --user

In [3]:
# Import of modules
import numpy as np  # Numerical operations
from pymatgen.core import Structure, Composition  # Materials analysis for crystal structures
import pandas as pd  # Data manipulation with DataFrames
import matminer  # Materials informatics
from matminer.datasets.dataset_retrieval import load_dataset  # Load materials datasets




# To make the model run faster
teaching_mode = True

In [ ]:
# Get dataset info from matminer
info = get_all_dataset_info("matbench_expt_gap")


# Check out the info about the dataset.
print(info)

Dataset: matbench_expt_gap
Description: Matbench v0.1 test dataset for predicting experimental band gap from composition alone. Retrieved from Zhuo et al. supplementary information. Deduplicated according to composition, removing compositions with reported band gaps spanning more than a 0.1eV range; remaining compositions were assigned values based on the closest experimental value to the mean experimental value for that composition among all reports. For benchmarking w/ nested cross validation, the order of the dataset must be identical to the retrieved data; refer to the Automatminer/Matbench publication for more details.
Columns:
	composition: Chemical formula.
	gap expt: Target variable. Experimentally measured gap, in eV.
Num Entries: 4604
Reference: Y. Zhuo, A. Masouri Tehrani, J. Brgoch (2018) Predicting the Band Gaps of Inorganic Solids by Machine Learning J. Phys. Chem. Lett. 2018, 9, 7, 1668-1673 https:doi.org/10.1021/acs.jpclett.8b00124.
Bibtex citations: ["@Article{Dunn2020

## Model choice

Define your model and justify your choice based on the problem and available data. You can look back at earlier notebooks and investigate other examples online including in [scikit-learn](https://scikit-learn.org).

In [12]:
df = load_dataset("matbench_expt_gap")


# df =  df.sample(1000)
df

,composition,gap expt
0,Ag(AuS)2,0.00
1,Ag(W3Br7)2,0.00
2,Ag0.5Ge1Pb1.75S4,1.83
3,Ag0.5Ge1Pb1.75Se4,1.51
4,Ag2BBr,0.00
...,...,...
4599,ZrTaN3,1.72
4600,ZrTe,0.00
4601,ZrTi2O,0.00
4602,ZrTiF6,0.00


In [ ]:
from matminer.featurizers.composition.element import BandCenter
from matminer.featurizers.composition.ion import IonProperty, ElectronegativityDiff, OxidationStates

from elementembeddings.composition import composition_featuriser, CompositionalEmbedding

from matminer.featurizers.composition.composite import ElementProperty
# # Sample composition data
# composition_data = {'Fe': 0.5, 'O': 0.5}

# # Convert composition data into a Pymatgen Composition object
# composition = Composition(composition_data)

# # Create an instance of the BandCenter featurizer
# bc = BandCenter()

# # Apply the featurizer to your composition data
# band_centers = bc.featurize(composition)

# print(band_centers)

# l = 0
# for i in df["composition"]:
#     try:
#         print(ElementProperty.from_preset(preset_name="magpie").featurize(Composition(CompositionalEmbedding(i, "magpie").fractional_composition)))
#     except KeyError:
#         l = l + 1
# print(l)

# df1 = pd.DataFrame()



# # Iterate over rows in the DataFrame
# for index, row in df.iterrows():
#     # Create a Composition object from the composition data
#     composition = Composition(row['composition'])
#     # Featurize the composition using CompositionalEmbedding with "magpie" preset
#     featurized_composition = CompositionalEmbedding(composition, "magpie").fractional_composition
#     # Store the featurized composition in the 'fractional_composition' column of df_featurised
#     df_featurised.at[index, 'fractional_composition'] = featurized_composition

# print(df_featurised)

# # # Adding column to include band centers
# # OxidationStates().featurize("ZrTiF6")

# from matminer.featurizers.composition import ElementProperty
# import pandas as pd

# # Sample composition DataFrame
# composition_df = pd.DataFrame({'composition': [{'Fe': 0.5, 'O': 0.5}, {'Al': 0.4, 'O': 0.3, 'N': 0.3}]})

# # Create an instance of the ElementProperty featurizer with ElectronAffinity property
# ep = ElementProperty.from_preset(preset_name="magpie")

# # Apply the featurizer to the composition DataFrame
# featurized_df = ep.featurize_dataframe(composition_df, col_id='composition')

# print(featurized_df)

# Function to calculate fractional composition

df = df.sample(1000)

# Apply the function to create the new column
df['fractional_composition'] = df['composition'].apply(lambda x: Composition(x).fractional_composition)
df["oxidation_states"] = df['composition'].apply(lambda x: Composition(x).add_charges_from_oxi_state_guesses())


# df1 = ElementProperty.from_preset(preset_name="magpie").featurize_dataframe(df, col_id='fractional_composition')
# df2 = BandCenter().featurize_dataframe(df, col_id='fractional_composition', ignore_errors=True)


df3 = OxidationStates().featurize_dataframe(df = df, col_id='oxidation_states', ignore_errors=True)



df3

ValueError: Charge balance analysis requires integer values in Composition!

In [13]:
# Featurising df
from elementembeddings.composition import composition_featuriser, CompositionalEmbedding
from matminer.featurizers.composition.composite import ElementProperty
from matminer.featurizers.composition.element import BandCenter

# df = df.sample(1000)

# Adding fractional composition column to main database.
df['fractional_composition'] = df['composition'].apply(lambda x: Composition(x).fractional_composition)

# Featurising based on magpie data using fractional composition.
df_featurised = ElementProperty.from_preset(preset_name="magpie").featurize_dataframe(df, col_id='fractional_composition')

# Featurising by adding one-hot collumns of each compound
composition_df = composition_featuriser(df["composition"], embedding="atomic", stats=["sum"])
df_featurised = pd.concat([df_featurised, composition_df], axis=1)

# Featurising by adding one-hot collumns of each compound
bandcenter_df = BandCenter().featurize_dataframe(df, col_id='fractional_composition', ignore_errors=True)
df_featurised = pd.concat([df_featurised, bandcenter_df], axis=1)

# Adding column that includes number of atoms in each compound
df_featurised["num_atoms"] = df["composition"].apply(lambda x: CompositionalEmbedding(x, "magpie").num_atoms)



# # Adding band centers
# for index, row in df.iterrows():
#      try:
#          df_featurised.at[index, 'band_center'] = BandCenter().featurize(Composition(CompositionalEmbedding(row.composition, "magpie").fractional_composition))
#      except KeyError:
#          df_featurised.at[index, 'band_center'] = np.nan



df_featurised

ElementProperty:   0%|          | 0/4604 [00:00<?, ?it/s]

Featurising compositions...


100%|██████████| 4604/4604 [00:06<00:00, 702.93it/s]


Computing feature vectors...


100%|██████████| 4604/4604 [00:00<00:00, 95106.78it/s]


BandCenter:   0%|          | 0/4604 [00:00<?, ?it/s]

,composition,gap expt,fractional_composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,...,sum_Fl,sum_Mc,sum_Lv,sum_Ts,sum_Og,composition,gap expt,fractional_composition,band center,num_atoms
0,Ag(AuS)2,0.00,"(Ag, Au, S)",16.0,79.0,63.0,47.400000,25.280000,16.0,65.0,...,0.0,0.0,0.0,0.0,0.0,Ag(AuS)2,0.00,"(Ag, Au, S)",5.640081,5.00
1,Ag(W3Br7)2,0.00,"(Ag, W, Br)",35.0,74.0,39.0,46.714286,15.619048,35.0,51.0,...,0.0,0.0,0.0,0.0,0.0,Ag(W3Br7)2,0.00,"(Ag, W, Br)",6.330247,21.00
2,Ag0.5Ge1Pb1.75S4,1.83,"(Ag, Ge, Pb, S)",16.0,82.0,66.0,36.275862,23.552913,16.0,65.0,...,0.0,0.0,0.0,0.0,0.0,Ag0.5Ge1Pb1.75S4,1.83,"(Ag, Ge, Pb, S)",5.198208,7.25
3,Ag0.5Ge1Pb1.75Se4,1.51,"(Ag, Ge, Pb, Se)",32.0,82.0,50.0,46.206897,17.388823,34.0,65.0,...,0.0,0.0,0.0,0.0,0.0,Ag0.5Ge1Pb1.75Se4,1.51,"(Ag, Ge, Pb, Se)",5.044300,7.25
4,Ag2BBr,0.00,"(Ag, B, Br)",5.0,47.0,42.0,33.500000,14.250000,47.0,65.0,...,0.0,0.0,0.0,0.0,0.0,Ag2BBr,0.00,"(Ag, B, Br)",5.032223,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4599,ZrTaN3,1.72,"(Zr, Ta, N)",7.0,73.0,66.0,26.800000,23.760000,7.0,44.0,...,0.0,0.0,0.0,0.0,0.0,ZrTaN3,1.72,"(Zr, Ta, N)",5.627749,5.00
4600,ZrTe,0.00,"(Zr, Te)",40.0,52.0,12.0,46.000000,6.000000,40.0,44.0,...,0.0,0.0,0.0,0.0,0.0,ZrTe,0.00,"(Zr, Te)",4.402714,2.00
4601,ZrTi2O,0.00,"(Zr, Ti, O)",8.0,40.0,32.0,23.000000,8.500000,22.0,43.0,...,0.0,0.0,0.0,0.0,0.0,ZrTi2O,0.00,"(Zr, Ti, O)",4.221115,4.00
4602,ZrTiF6,0.00,"(Zr, Ti, F)",9.0,40.0,31.0,14.500000,8.250000,9.0,43.0,...,0.0,0.0,0.0,0.0,0.0,ZrTiF6,0.00,"(Zr, Ti, F)",7.923077,8.00


In [ ]:
for i in (df1.columns):
    print(i)

composition
gap expt
fractional_composition
MagpieData minimum Number
MagpieData maximum Number
MagpieData range Number
MagpieData mean Number
MagpieData avg_dev Number
MagpieData mode Number
MagpieData minimum MendeleevNumber
MagpieData maximum MendeleevNumber
MagpieData range MendeleevNumber
MagpieData mean MendeleevNumber
MagpieData avg_dev MendeleevNumber
MagpieData mode MendeleevNumber
MagpieData minimum AtomicWeight
MagpieData maximum AtomicWeight
MagpieData range AtomicWeight
MagpieData mean AtomicWeight
MagpieData avg_dev AtomicWeight
MagpieData mode AtomicWeight
MagpieData minimum MeltingT
MagpieData maximum MeltingT
MagpieData range MeltingT
MagpieData mean MeltingT
MagpieData avg_dev MeltingT
MagpieData mode MeltingT
MagpieData minimum Column
MagpieData maximum Column
MagpieData range Column
MagpieData mean Column
MagpieData avg_dev Column
MagpieData mode Column
MagpieData minimum Row
MagpieData maximum Row
MagpieData range Row
MagpieData mean Row
MagpieData avg_dev Row
Magp

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the features we want in the DataFrame
feature_cols = [col for col in list(df_featurised.columns) if col != "formula"]

# Get an array of the features
prelim_X = df_featurised[feature_cols]

# Get an array of the target variable
y = df["gap expt"]

# Training a preliminary random forest regressor to understand the importance of different features
prelim_rf = RandomForestRegressor()
prelim_rf.fit(prelim_X, y)

RandomForestRegressor()

In [ ]:
import plotly.express as px

In [ ]:
# Import the class
from elementembeddings.core import Embedding

# Load the atomic data
atomic_feature_labels = Embedding.load_data('atomic').feature_labels
one_hot_columns = ["sum_" + a for a in atomic_feature_labels]


# Get index of all one-hot encoded element columns to sum their feature importances
imp = prelim_rf.feature_importances_
feature_col = prelim_X.columns

elem_indices = [np.where(np.isin(prelim_X.columns, one_hot_columns))[0]]
elem_imp = imp[np.where(np.isin(prelim_X.columns, one_hot_columns))[0]].sum()

# Removing individual element contributions and adding them together
imp = np.delete(imp, elem_indices)
imp = np.append(imp, elem_imp)

# Removing individual element columns and adding "Elements"
feature_col = np.delete(feature_col, elem_indices)
feature_col = np.append(feature_col, "Elements")

# Sorting both arrays based on descending order of importances
sort_inds = imp.argsort()
imp = imp[sort_inds[::-1]]
feature_col = feature_col[sort_inds[::-1]]

In [ ]:
fig = px.bar(x = feature_col, y = imp,
             labels={"x": "Preliminary Feature Columns", "y": "Feature Importances"},
             title="Feature Importance Bar Chart")
fig.show()

In [ ]:
feature_df = pd.DataFrame()
feature_df["Columns"] = prelim_X.columns
feature_df["Feature Importance"] = prelim_rf.feature_importances_

# filtered_columns = prelim_X.columns[prelim_X.columns.isin(one_hot_columns)]

feature_df["Feature Importance"] = feature_df.loc[feature_df['Columns'].isin(one_hot_columns), 'Feature Importance'].sum()

# feature_df

0.1454448869866831

0.1454448869866831

In [ ]:


# df_featurised[one_hot_columns]

In [ ]:
# prelim_rf.feature_importances_
# for i in prelim_X.columns:
#     print (i)


# Get index of all atomic features to sum all feature importances regarding the same.


mean_Number
mean_MendeleevNumber
mean_AtomicWeight
mean_MeltingT
mean_Column
mean_Row
mean_CovalentRadius
mean_Electronegativity
mean_NsValence
mean_NpValence
mean_NdValence
mean_NfValence
mean_NValence
mean_NsUnfilled
mean_NpUnfilled
mean_NdUnfilled
mean_NfUnfilled
mean_NUnfilled
mean_GSvolume_pa
mean_GSbandgap
mean_GSmagmom
mean_SpaceGroupNumber
sum_Number
sum_MendeleevNumber
sum_AtomicWeight
sum_MeltingT
sum_Column
sum_Row
sum_CovalentRadius
sum_Electronegativity
sum_NsValence
sum_NpValence
sum_NdValence
sum_NfValence
sum_NValence
sum_NsUnfilled
sum_NpUnfilled
sum_NdUnfilled
sum_NfUnfilled
sum_NUnfilled
sum_GSvolume_pa
sum_GSbandgap
sum_GSmagmom
sum_SpaceGroupNumber
sum_H
sum_He
sum_Li
sum_Be
sum_B
sum_C
sum_N
sum_O
sum_F
sum_Ne
sum_Na
sum_Mg
sum_Al
sum_Si
sum_P
sum_S
sum_Cl
sum_Ar
sum_K
sum_Ca
sum_Sc
sum_Ti
sum_V
sum_Cr
sum_Mn
sum_Fe
sum_Co
sum_Ni
sum_Cu
sum_Zn
sum_Ga
sum_Ge
sum_As
sum_Se
sum_Br
sum_Kr
sum_Rb
sum_Sr
sum_Y
sum_Zr
sum_Nb
sum_Mo
sum_Tc
sum_Ru
sum_Rh
sum_Pd
sum_Ag
sum_

In [ ]:
import elementembeddings


# # Fe2O3_magpie = elementembeddings.composition.CompositionalEmbedding.num_atoms
# df2 = CompositionalEmbedding(df["composition"], "magpie").num_atoms
df_featurised

,formula,mean_Number,mean_MendeleevNumber,mean_AtomicWeight,mean_MeltingT,mean_Column,mean_Row,mean_CovalentRadius,mean_Electronegativity,mean_NsValence,...,sum_NsUnfilled,sum_NpUnfilled,sum_NdUnfilled,sum_NfUnfilled,sum_NUnfilled,sum_GSvolume_pa,sum_GSbandgap,sum_GSmagmom,sum_SpaceGroupNumber,num_atoms
0,Ag(AuS)2,47.400000,74.600000,113.186268,937.262000,13.000000,4.600000,125.400000,2.434000,1.400000,...,3.0,4.0,0.0,0.0,7.0,101.303750,4.404,0.000000,815.00,5.00
1,Ag(W3Br7)2,46.714286,81.000000,110.931629,1291.720476,13.571429,4.619048,133.190476,2.739524,1.952381,...,1.0,14.0,36.0,0.0,51.0,525.350000,20.398,0.000000,2495.00,21.00
2,Ag0.5Ge1Pb1.75S4,36.275862,83.482759,85.163324,611.499655,14.896552,4.000000,119.724138,2.396207,1.931034,...,0.5,19.0,0.0,0.0,19.5,183.510000,9.191,0.000000,1011.25,7.25
3,Ag0.5Ge1Pb1.75Se4,46.206897,84.034483,111.036428,669.783793,14.896552,4.551724,128.000000,2.379655,1.931034,...,0.5,19.0,0.0,0.0,19.5,184.042500,3.579,0.000000,787.25,7.25
4,Ag2BBr,33.500000,74.250000,76.612850,1270.915000,13.000000,4.000000,123.500000,2.215000,1.500000,...,2.0,6.0,0.0,0.0,8.0,69.312500,2.981,0.000000,680.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4599,ZrTaN3,26.800000,67.600000,62.838396,1121.430000,10.800000,3.400000,111.600000,2.390000,2.000000,...,0.0,9.0,15.0,0.0,24.0,85.621250,19.311,0.000000,1005.00,5.00
4600,ZrTe,46.000000,67.000000,109.412000,1425.330000,10.000000,5.000000,156.500000,1.715000,2.000000,...,0.0,2.0,8.0,0.0,10.0,57.958333,0.464,0.000000,346.00,2.00
4601,ZrTi2O,23.000000,54.250000,50.739350,1516.200000,7.000000,3.750000,140.250000,1.962500,2.000000,...,0.0,2.0,24.0,0.0,26.0,65.680000,0.000,0.000045,594.00,4.00
4602,ZrTiF6,14.500000,80.625000,31.635177,548.750000,13.750000,2.625000,84.625000,3.343750,2.000000,...,0.0,6.0,16.0,0.0,22.0,98.130000,11.820,0.000023,478.00,8.00


In [ ]:
from CBFV import composition

X, y, formulae, skipped = composition.generate_features(df2,
                                                        elem_prop='magpie',
                                                        drop_duplicates=False,
                                                        extend_features=True,
                                                        sum_feat=False)

X

Processing Input Data: 100%|██████████| 4604/4604 [00:00<00:00, 13087.00it/s]


	Featurizing Compositions...


Assigning Features...: 100%|██████████| 4604/4604 [00:00<00:00, 6562.10it/s]


	Creating Pandas Objects...


,avg_Number,avg_MendeleevNumber,avg_AtomicWeight,avg_MeltingT,avg_Column,avg_Row,avg_CovalentRadius,avg_Electronegativity,avg_NsValence,avg_NpValence,...,mode_NValence,mode_NsUnfilled,mode_NpUnfilled,mode_NdUnfilled,mode_NfUnfilled,mode_NUnfilled,mode_GSvolume_pa,mode_GSbandgap,mode_GSmagmom,mode_SpaceGroupNumber
0,47.400000,74.600000,113.186268,937.262000,13.000000,4.600000,125.400000,2.434000,1.400000,1.600000,...,6.0,0.0,0.0,0.0,0.0,1.0,16.700000,0.000,0.000000,70.0
1,46.714286,81.000000,110.931629,1291.720476,13.571429,4.619048,133.190476,2.739524,1.952381,3.333333,...,17.0,0.0,1.0,0.0,0.0,1.0,29.480000,1.457,0.000000,64.0
2,36.275862,83.482759,85.163324,611.499655,14.896552,4.000000,119.724138,2.396207,1.931034,2.965517,...,6.0,0.0,2.0,0.0,0.0,2.0,25.786875,2.202,0.000000,70.0
3,46.206897,84.034483,111.036428,669.783793,14.896552,4.551724,128.000000,2.379655,1.931034,2.965517,...,16.0,0.0,2.0,0.0,0.0,2.0,25.920000,0.799,0.000000,14.0
4,33.500000,74.250000,76.612850,1270.915000,13.000000,4.000000,123.500000,2.215000,1.500000,1.500000,...,11.0,1.0,0.0,0.0,0.0,1.0,16.330000,0.000,0.000000,225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4599,26.800000,67.600000,62.838396,1121.430000,10.800000,3.400000,111.600000,2.390000,2.000000,1.800000,...,5.0,0.0,3.0,0.0,0.0,3.0,14.768750,6.437,0.000000,194.0
4600,46.000000,67.000000,109.412000,1425.330000,10.000000,5.000000,156.500000,1.715000,2.000000,2.000000,...,4.0,0.0,0.0,0.0,0.0,2.0,23.195000,0.000,0.000000,152.0
4601,23.000000,54.250000,50.739350,1516.200000,7.000000,3.750000,140.250000,1.962500,2.000000,1.000000,...,4.0,0.0,0.0,8.0,0.0,8.0,16.690000,0.000,0.000023,194.0
4602,14.500000,80.625000,31.635177,548.750000,13.750000,2.625000,84.625000,3.343750,2.000000,3.750000,...,7.0,0.0,1.0,0.0,0.0,1.0,9.707500,1.970,0.000000,15.0


## Training and testing

Train, validate and test your model. Comment on the performance metrics. Note the best-performing model on the [MatBench](https://matbench.materialsproject.org) leaderboard.  With limited resources, don't expect to match this performance, but you should do better than a baseline model.

<details>
<summary>Note on the ROC-AUC classification metric</summary>
There is one metric we didn't cover but is used in Matbench. In binary classification models, the ROC-AUC (Receiver Operating Characteristic - Area Under the Curve) score can be used to evaluate performance. It quantifies the ability of the model to distinguish between positive and negative instances across different decision thresholds. A higher ROC-AUC score (ranging from 0.5 to 1) indicates better performance, with 1 representing a perfect classifier and 0.5 indicating performance no better than random chance. There is a more detailed discussion on https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc.

The metric can be calculated using the `roc_auc_score` function from the `sklearn.metrics` module, e.g.

```python
from sklearn.metrics import roc_auc_score

# Assuming you have true labels (y_true) and predicted probabilities (y_pred_prob)
y_true = [...]  
y_pred_prob = [...]  

# Calculate ROC-AUC
roc_auc = roc_auc_score(y_true, y_pred_prob)

# Display the result
print(f'ROC-AUC Score: {roc_auc:.4f}')
```
</details>

In [ ]:
# Spare cell




## Large Language Model (LLM) usage declaration

You should acknowledge any use of a generative model during your assignment. Points to consider:

* State which LLM (e.g. GPT-3, Gemini, Co-Pilot)

* Specify tasks (e.g. summarising research or code snippets)

* Were any limitations/biases noted?

* How did you ensure ethical use?

In [ ]:
# Spare cell




## ☘️ Final word

Good luck building your own model! I hope that you enjoyed the course and exercises. Dive deeper into the aspects that caught your interest. A useful starting point may be the [Resources](https://aronwalsh.github.io/MLforMaterials/Resources.html) page.

Remember that submission is on Blackboard and this time you should upload the actual Juypter Notebook (`.ipynb` file), as well as your recorded narrated presentation (maximum 5 minutes; see guides on using [Zoom](https://www.youtube.com/watch?v=H9qhoAIzW3E) or [Powerpoint](https://www.youtube.com/watch?v=Y5dgwwa5XRA) for this purpose).